### Accelerate memory estimates

In [1]:
!accelerate estimate-memory ibm-granite/granite-7b-base

Loading pretrained config for `ibm-granite/granite-7b-base` from `transformers`...
/home/shanand/ilab/lib64/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
┌────────────────────────────────────────────────────┐
│Memory Usage for loading `ibm-granite/granite-7b-base`│
├───────┬─────────────┬──────────┬───────────────────┤
│ dtype │Largest Layer│Total Size│Training using Adam│
├───────┼─────────────┼──────────┼───────────────────┤
│float32│  774.03 MB  │ 24.68 GB │      98.71 GB     │
│float16│  387.02 MB  │ 12.34 GB │      49.35 GB     │
│  int8 │  193.51 MB  │ 6.17 GB  │      24.68 GB     │
│  int4 │   96.75 MB  │ 3.08 GB  │      12.34 GB     │
└───────┴─────────────┴──────────┴───────────────────┘


In [2]:
!accelerate estimate-memory databricks/dolly-v2-3b

Loading pretrained config for `databricks/dolly-v2-3b` from `transformers`...
/home/shanand/ilab/lib64/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
┌────────────────────────────────────────────────────┐
│ Memory Usage for loading `databricks/dolly-v2-3b`  │
├───────┬─────────────┬──────────┬───────────────────┤
│ dtype │Largest Layer│Total Size│Training using Adam│
├───────┼─────────────┼──────────┼───────────────────┤
│float32│  491.02 MB  │ 9.88 GB  │      39.54 GB     │
│float16│  245.51 MB  │ 4.94 GB  │      19.77 GB     │
│  int8 │  122.75 MB  │ 2.47 GB  │      9.88 GB      │
│  int4 │   61.38 MB  │ 1.24 GB  │      4.94 GB      │
└───────┴─────────────┴──────────┴───────────────────┘


In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, AdamW, get_scheduler
from transformers import DataCollatorWithPadding
from tqdm.auto import tqdm
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
import torch

/home/shanand/ilab/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "databricks/dolly-v2-3b"
#model_path = "ibm-granite/granite-3b-code-base"
#model_path = "ibm-granite/granite-7b-base"



In [3]:
model = AutoModelForCausalLM.from_pretrained(model_path)
#optimizer = AdamW(model.parameters(), lr=3e-5)

import bitsandbytes as bnb
optimizer = bnb.optim.Adam8bit(model.parameters(), lr=1e-3)

/home/shanand/ilab/lib64/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_prompt(data_point):
    prompt = f""" Given a question and the possible answer choices, give the index of the right choice.
    ### Question
    {data_point['question']}
    ### Choices
    {data_point['choices']}
    ### Answer
    {data_point['answer']}
    """
    result = tokenizer(prompt,
                     padding="max_length",
                     max_length=512,
                     truncation=True)
    # Why is this necessary?
    result["labels"] = result["input_ids"].copy()
    return result

dataset = load_dataset("cais/mmlu", 'all')
small_train_dataset = dataset['auxiliary_train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset['validation'].shuffle(seed=42).select(range(200))

# I see a very weird behavior (scalar input IDs instead of max length vector) when I do batched=True in the map funciton
# Anyhow that's only for speeding up the map task
small_train_dataset_tokenized = small_train_dataset.map(tokenize_prompt,
                                                        remove_columns=['question', 'subject', 'choices', 'answer'])
small_eval_dataset_tokenized = small_eval_dataset.map(tokenize_prompt,
                                                      remove_columns=['question', 'subject', 'choices', 'answer'])

small_train_dataset_tokenized.set_format("torch")
small_eval_dataset_tokenized.set_format("torch")

data_collator = DataCollatorForLanguageModeling(tokenizer,
                                               mlm=False)

train_dataloader = DataLoader(small_train_dataset_tokenized, shuffle=True, batch_size=2, collate_fn=data_collator)
eval_dataloader = DataLoader(small_eval_dataset_tokenized, batch_size=2, collate_fn=data_collator)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
accelerator = Accelerator(mixed_precision="fp16")

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)


In [7]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
eval_interval = 100
global_step = 0

progress_bar = tqdm(range(num_training_steps))
model.gradient_checkpointing_enable()
model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        print(loss)
        accelerator.backward(loss)
        accelerator.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        global_step += 1
        
        if global_step % eval_interval == 0:
            model.eval()
            eval_loss = 0
            for eval_batch in eval_dl:
                with torch.no_grad():
                    eval_outputs = model(**eval_batch)
                    eval_loss += eval_outputs.loss.item()
            eval_loss /= len(eval_dl)
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{global_step}], Eval Loss: {eval_loss}")
            
            model.train()

tensor(5.3460, device='cuda:0', grad_fn=<NllLossBackward0>)


 64%|██████▎   | 956/1500 [08:28<04:06,  2.21it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


tensor(6.0163, device='cuda:0', grad_fn=<NllLossBackward0>)


 64%|██████▍   | 957/1500 [08:28<04:06,  2.21it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


tensor(8.3863, device='cuda:0', grad_fn=<NllLossBackward0>)


...

tensor(5.4624, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|█████████▉| 1499/1500 [13:16<00:00,  2.22it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


tensor(5.5589, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 1500/1500 [13:16<00:00,  2.22it/s]

Epoch [3/3], Step [1500], Eval Loss: 5.842406475543976


In [8]:
model.eval()
eval_loss = 0
for eval_batch in eval_dl:
    with torch.no_grad():
        eval_outputs = model(**eval_batch)
        eval_loss += eval_outputs.loss.item()
eval_loss /= len(eval_dl)
print(f"Epoch [{epoch+1}/{num_epochs}], Step [{global_step}], Eval Loss: {eval_loss}")

Epoch [3/3], Step [1500], Eval Loss: 5.842406475543976


In [ ]:
## Exploding gradients

In [11]:
model.eval()
eval_loss = 0
for eval_batch in train_dl:
    with torch.no_grad():
        eval_outputs = model(**eval_batch)
        eval_loss += eval_outputs.loss.item()
    break
        #print(eval_outputs.loss.item())

eval_loss /= len(eval_dl)
eval_loss

## This works too so it's not the train_dl as well

0.02767714262008667

In [9]:
model.train()
train_loss = 0
for train_batch in train_dl:
    outputs = model(**train_batch)
    train_loss = outputs.loss.item()
    break
        #print(eval_outputs.loss.item())

train_loss

## THis works too. It's basically a problem with the model.gradient_checkpointing
##  but without that, training doesn't fit on the gpu

3.0101101398468018

In [10]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/8a/5d/047cde25131eef3a38d03317fa7d25d6f60ce6e8ccfd24ac88b3e309fc00/scikit_learn-1.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/9b/00/ce54410e344b3a6032cd42ed53fe425cf57a66d28e337670292bbb419ebc/scipy-1.14.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 499.1 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.me

In [26]:
batch["labels"].cpu().numpy().shape

(2, 512)

### Testing

In [28]:
from sklearn.metrics import accuracy_score

model.eval()
predictions, references = [], []

for batch in eval_dl:
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
    decoded_preds = tokenizer.batch_decode(preds.cpu().numpy(), skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(batch["input_ids"].cpu().numpy(), skip_special_tokens=True)

    predictions.extend(decoded_preds)
    references.extend(decoded_labels)


In [33]:
references[0]

" Given a question and the possible answer choices, give the index of the right choice.\n    ### Question\n    Lewis Henry Morgan's stages of human culture are known as:\n    ### Choices\n    ['Three-Age system.', 'Natural selection.', 'Multilineal evolution.', 'Unilineal evolution.']\n    ### Answer\n    3\n    "

In [42]:
print(predictions[6])

 a question and the possible answer choices, give the index of the right choice.
    ### Question
    ###.. to the right.. The     the right. the...I.. the.. the the have.. the the the the. a the the right. the right be. the right be. the the. the right. the the right be. the. a. right.. the the school to. a to the. a right the... the a... the to a the.. the. the the and. a the the to the right. the. the first... the have.. the. the.. a the right.I.. the.. the. the to the to the the first and.. to the.. the be of The be.. the right..... the the the.. the right... be to school be of the right. the be... the to. the the and... the right.. the right to. The   . be. the school. the and was.... the. the..   . the right. and. right.. the. the right. to
    ### Choices
    ###'t first. be the the right... to's.. right.. ''t first. be the school... to's.. right.. ''t first. to the be... ''t first. to right.. the right and. the right..'t
    ### Cho
    ###
    ###..............................

## Deepspeed world

In [ ]:
from deepspeed.runtime.zero.stage3 import estimate_zero3_model_states_mem_needs_all_live
estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=2, num_nodes=1)